<table>
  <tr>
    <td style="border: none; padding-right: 20px;">
      <img src="logo_uni.png" width="300" style="margin: 0; padding: 0;">
    </td>
    <td style="border: none; vertical-align: middle;">
      <h1 style="margin: 0;">Airline Customer Segmentation</h1>
      <p style="margin: 0; font-size: 18px;">Master in Data Science - 2025/26</p>
      <p style="margin: 0; font-style: italic;">Developed by: <b>[Alexandre Coelho, 20250475], [Bruna Sousa, 20250526], [Rui Ferreira, 20250473] 
</b></p>
    </td>
  </tr>
</table>

In [ ]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.colors as mpl_colors
from narwhals.stable.v1 import Datetime
from sklearn.cluster import KMeans, HDBSCAN, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import functions as func
from scipy.cluster.hierarchy import dendrogram
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from minisom import MiniSom
import scipy.cluster.hierarchy as sch
import plotly.express as px


In [ ]:
try:
    from functions import *
    print("SUCCESS: Custom functions loaded correctly from 'my_functions.py'.")
except ImportError:
    print("ERROR: Could not find 'my_functions.py'. Make sure the file is in the same folder as this notebook.")

## Table of Contents
- [Business Understanding](#business-understanding)
- [Data Understanding](#data-understanding)
  - [Customer Database](#customer-database)
    - [Reading the data](#reading-the-data)
    - [Metadata](#metadata)
    - [Initial Analysis](#initial-analysis)
    - [Data Quality Checks](#data-quality-checks)
  - [Flights Database](#flights-database)
    - [Reading the data](#reading-the-data-1)
    - [Metadata](#metadata-1)
    - [Initial Analysis](#initial-analysis-1)
    - [Data Quality Checks](#data-quality-checks-1)
- [Data Preparation](#data-preparation)
  - [Customers Database](#customers-database)
    - [Data Types](#data-types)
    - [Duplicates](#duplicates)
    - [Missing Values](#missing-values)
    - [Coherence Check](#coherence-check)
  - [Flights Database](#flights-database-1)
    - [Data Types](#data-types-1)
    - [Duplicates](#duplicates-1)
    - [Coherence Check](#coherence-check-1)
    - [Coherence Check on both databases](#coherence-check-on-both-databases)
- [Data Exploration](#data-exploration)
  - [Customer](#customer)
  - [Distributions of Categorical Features](#distributions-of-categorical-features)
  - [Distribution of Numerical Features](#distribution-of-numerical-features)

## 1. Business Understanding

AIAI aims to use customer data to understand their behaviors, with the goal of improving satisfaction, customer value, and profitability. The main challenge is to segment customers based on their travel habits and loyalty, enabling the creation of personalized marketing and optimized services. The project aims to transform these analytical findings into actionable business strategies.

## 2. Data Understanding

### 2.1 Customer Database

#### 2.1.1 Reading the data

In [ ]:
customer_data = pd.read_csv("DM_AIAI_CustomerDB.csv",index_col=0)

#### 2.1.2 Metadata
- *Loyalty#* - Unique customer identifier for loyalty program members
- *First Name* - Customer's first name
- *Last Name* -	Customer's last name
- *Customer Name* - Customer's full name (concatenated)
- *Country* - Customer's country of residence
- *Province or State* - Customer's province or state
- *City* - Customer's city of residence
- *Latitude* - Geographic latitude coordinate of customer location
- *Longitude* - Geographic longitude coordinate of customer location
- *Postal code* - Customer's postal/ZIP code
- *Gender* - Customer's gender
- *Education* - Customer's highest education level (Bachelor, College, etc.)
- *Location Code*- Urban/Suburban/Rural classification of customer residence
- *Income* - Customer's annual income
- *Marital Status* - Customer's marital status (Married, Single, Divorced)
- *LoyaltyStatus* - Current tier status in loyalty program (Star > Nova > Aurora)
- *EnrollmentDateOpening* - Date when customer joined the loyalty program
- *CancellationDate* - Date when customer left the program
- *Customer Lifetime Value* - Total calculated monetary value of customer relationship
- *EnrollmentType* - Method of joining loyalty program


#### 2.1.3 Initial Analysis
General idea of the data structure, column names and types of values each variable contains.

In [ ]:
customer = customer_data.copy()
customer.head(3)

In [ ]:
print(f"Numbers of lines and columns: {customer.shape}")

In [ ]:
customer.info()

In [ ]:
customer.describe(include="all").T

#### 2.1.4 Data Quality Checks

In [ ]:
customer.replace("", np.nan, inplace=True)
customer.isna().sum()

We have 20 customers without income data, 20 customers without Customer Lifetime Value data and 14611 customers without CancellationDate data.

In [ ]:
customer.duplicated().sum()

We have no duplicated rows in our data.

### 2.2 Flights Database

#### 2.2.1 Reading the data

In [ ]:
flight_data = pd.read_csv("DM_AIAI_FlightsDB.csv")

#### 2.2.2 Metadata
- *Variable* - Description
- *Loyalty#* - Unique customer identifier linking to CustomerDB
- *Year* -	Year of flight activity record
- *Month* - Month of flight activity record (1-12)
- *YearMonthDate* - First day of the month for the activity period
- *NumFlights* - Total number of flights taken by customer in the month
- *NumFlightsWithCompanions* - Number of flights where customer traveled with companions
- *DistanceKM* - Total distance traveled in kilometers for the month
- *PointsAccumulated* - Loyalty points earned by customer during the month
- *PointsRedeemed* - Loyalty points spent/redeemed by customer during the month
- *DollarCostPointsRedeemed* - Dollar value of points redeemed during the month

#### 2.2.3 Initial Analysis

General idea of the data structure, column names and types of values each variable contains.

In [ ]:
flight = flight_data.copy()
flight.head(3)

In [ ]:
print(f"Numbers of lines and columns: {flight.shape}")

In [ ]:
flight.info()

In [ ]:
flight.describe(include="all").T

#### 2.2.4 Data Quality Checks

In [ ]:
flight.replace("", np.nan, inplace=True)
flight.isna().sum()

We have no missing data.

In [ ]:
flight.duplicated().sum()

We have 2903 duplicated rowns in our data, that we will look at later.

## 3. Data Preparation

### 3.1 Customers Database

#### 3.1.1 Data Types

In [ ]:
customer.dtypes

The Enrollment Date Opening and Cancellation Dates can be converted to datetime type.

In [ ]:
customer["EnrollmentDateOpening"] = pd.to_datetime(customer["EnrollmentDateOpening"])
customer["CancellationDate"] = pd.to_datetime(customer["CancellationDate"], errors="coerce")

#### 3.1.2 Duplicates

To use the 'Loyalty#' column as the dataframe's index, it was essential to first confirm that it contained only unique values. An index acts as a primary key for data retrieval, and duplicate entries would make this unreliable.

Upon inspection, some 'Loyalty#' IDs were found to be duplicated. Since there was no clear way to determine which entry represented the true customer record (the "source of truth"), the most robust decision was to remove all records associated with these ambiguous IDs. This data cleansing step ensures the integrity of the dataset, leaving only customers with a single, unique 'Loyalty#' identifier, which can now be reliably set as the index.

In [ ]:
duplicated_loyalty = customer_data.duplicated(subset=['Loyalty#'], keep=False)
duplicated_loyalty_list = customer_data[duplicated_loyalty]['Loyalty#'].unique()
print(f"Duplicated Loyaltys {len(duplicated_loyalty_list)}")
customer = customer[customer['Loyalty#'].isin(duplicated_loyalty_list) == False]
print(f"For the initial customer data we have {len(customer_data)} records.\n"
      f"We identified {len(duplicated_loyalty_list)} 'Loyalty#' IDs that were duplicated or appears repeatedly.\n"
      f"So the final len of the data set is {len(customer)}.")

In [ ]:
customer.set_index("Loyalty#", inplace=True)

#### 3.1.3 Missing Values

In [ ]:
customer[customer["Income"].isna()].tail(3)

The customers with missing income information also have a Customer Lifetime Value equal to 0, which we believe is because their cancellation date is the same as their enrollment date, and it has no value for the company.

In [ ]:
customer_NaN_income = customer["Income"].isna().mean() * 100
print(f"The percentage of customers with missing data about income is {customer_NaN_income:.2f} %.")

Because the percentage of missing data about income is 0.12 %, it is safe to remove this rows.

In [ ]:
customer = customer.dropna(subset=['Income'])


#### 3.1.4 Coherence Check

In [ ]:
# Number of customers with available values in cancellation date
total_cancel_dates = len(customer[customer["CancellationDate"].notna()])
print(f"Number of available cancellation dates: {total_cancel_dates} ({((total_cancel_dates/len(customer))*100):.2f}%)")

Missing values in the CancellationDate column (which affects 86.45% of the customer base) are assumed to represent customers who have never officially terminated their loyalty membership. 

In [ ]:
# Number of customers with cancelation date before the enrollment date
incoherent_cancel_dates = len(customer[customer['CancellationDate'] < customer['EnrollmentDateOpening']])
print(f"Number of incoherent dates: {incoherent_cancel_dates} ({((incoherent_cancel_dates/total_cancel_dates)*100):.2f}%)")

The 199 incoherent dates (CancellationDate < EnrollmentDateOpening) strongly suggest a business scenario where members re-enrolled in the loyalty program. It is possible that the CancellationDate reflects an old membership end,  while EnrollmentDateOpening records the start of their current re-activated membership.

This group represents a valuable segment of returned customers,  which we should give special attention during the segmentation phase. 

In [ ]:
# Number of incoherent enrollment type 
cust_2021_promo = customer[customer["EnrollmentType"] == "2021 Promotion"].copy()
cust_2021_promo["EnrollmentYear"] = cust_2021_promo["EnrollmentDateOpening"].dt.year
incoherent_enrollment_type = cust_2021_promo[cust_2021_promo["EnrollmentYear"] !=2021.0]
print(f"Number of incoherent enrollment type (enrollment out of 2021): {len(incoherent_enrollment_type)}")

This check was performed to verify if the categorical feature "2021 Promotion" in EnrollmentType was chronologically coherent with the customer's EnrollmentDateOpening. The result shows that 172 of customers tagged with the "2021 Promotion" were registered in earlier years (2015, 2017,...).

The EnrollmentType variable remains useful for demographic/psychographic segmentation (identifying customers sensitive to incentives), but its value cannot be strictly relied upon for calculating customer longevity.

In [ ]:
# Number of customer's names that don´t correspond to concatenation of first name and last name
coherent_names = ( customer["First Name"].str.strip()+" "+customer["Last Name"].str.strip())
incoherent_names_condition = (customer["Customer Name"].str.strip() != coherent_names.str.strip())
incoherent_names = customer[incoherent_names_condition]
print(f"Number of unmatched customer names: {len(incoherent_names)}")

Furthermore, comparing Customer Name against the concatenation of First Name and Last Name confirmed zero inconsistencies, demonstrating the perfect integrity of the customer identification fields.

### 3.2 Flights Database

#### 3.2.1 Data Types

In [ ]:
flight.dtypes

We decided to change the data types of variables NumFlights, NumFlightsWithCompanions, PointsAccumulated and PointsRedeemed to integers because these variables are discrete.
Also, the YearMonthDate variable can be converted to datetime type.

In [ ]:
flight['YearMonthDate'] = pd.to_datetime(flight['YearMonthDate'], errors='coerce')

columns_to_convert = ["NumFlights", "NumFlightsWithCompanions", "PointsAccumulated", "PointsRedeemed"]
flight[columns_to_convert] = flight[columns_to_convert].astype(int)
print(flight[columns_to_convert].dtypes)

#### 3.2.2 Duplicates

Following the removal of ambiguous 'Loyalty#' IDs from the customer dataset, the same filtering was applied to the flight dataset. This step was crucial for maintaining referential integrity, as it ensures that the flight data only contains records for customers who are verifiably present in the customer data.

In [ ]:
flight = flight[flight['Loyalty#'].isin(duplicated_loyalty_list) == False]

In [ ]:
flight.duplicated().sum()

Interestingly, this action also resolved all duplicate issues within the flight dataset, indicating that the duplicated entries were tied to the same problematic 'Loyalty#' IDs. This clean-up ensures both datasets are now consistent and aligned.

#### 3.2.3 Coherence Checks

In [ ]:
# Number of flights uncoherent
print(f"Number of uncoherent number of flights: {len(flight[flight["NumFlightsWithCompanions"] > flight["NumFlights"]])}")

# Number of flights with uncoherent data of DistanceKM
print(f"Number of uncoherent number of KMs: {len(flight[flight["DistanceKM"] < flight["NumFlights"]])}")

# Number of points accumulated with 0 KM travelled
incoherent_points_condition = (flight["PointsAccumulated"] >0) & (flight["DistanceKM"] == 0)
incoherent_points = flight[incoherent_points_condition]
print(f"Number of incoherent points accumulated: {len(incoherent_points)}")

# Number of incoherent points redeemed or dollar cost points redeemed
incoherent_dollars_condition = (flight["PointsRedeemed"]>0) & (flight["DollarCostPointsRedeemed"]==0) | (flight["PointsRedeemed"]==0) & (flight["DollarCostPointsRedeemed"]>0)
incoherent_dollars = flight[incoherent_dollars_condition]
print(f"Number of incoherent dollars redeemed: {len(incoherent_dollars)}")

The flight activity data demonstrates high internal coherence, with zero records showing logical inconsistencies between flight counts, distance traveled, point accumulation and dollar cost of points redeemed.

### 3.3 Coherence Check on both databases

In [ ]:
# Number of incoherent customers flying after customer cancelling the membership
cust_cancelled = customer[customer["CancellationDate"].notna()].copy()

merged_flights_cust_cancel = pd.merge(
    customer[customer["CancellationDate"].notna()][["CancellationDate", "EnrollmentDateOpening"]],
    flight[["Loyalty#", "YearMonthDate", "NumFlights"]],
    left_index=True, right_on= "Loyalty#", how="inner"
)

flights_condition1 = (
    (merged_flights_cust_cancel["YearMonthDate"]> merged_flights_cust_cancel["CancellationDate"]) &
    (merged_flights_cust_cancel["NumFlights"]>0) &
    (merged_flights_cust_cancel["EnrollmentDateOpening"]<=merged_flights_cust_cancel["CancellationDate"])
)

flights1 = merged_flights_cust_cancel[flights_condition1]
print(f"Number of incoherent customers flying after customer cancelling the membership: {flights1["Loyalty#"].nunique()}")

This test verified if any customer registered flight activity after their official cancellation date without having subsequently re-enrolled in the loyalty program. The results showed zero unique customers exhibiting this true logical incoherence. The previously observed post-cancellation activity is fully justified by customer re-enrollment, meaning the old CancellationDate simply serves as a historical record rather than a current status flag.

In [ ]:
customer["CancellationDate"] = customer["CancellationDate"].fillna(pd.Timestamp("1904-01-01"))

dizer que agora sim mudamos o type do cancellation date 

## 4. Data Exploration

### 4.1 Customer Database

#### 4.1.1 Distribution of Categorical Features

In [ ]:

categorical_customer = ["Gender", "Education", "Marital Status", "LoyaltyStatus", "EnrollmentType", "Location Code","Country"]

plt.figure(figsize=(20, 12))
for i, col in enumerate(categorical_customer, 1):
    plt.subplot(2, 4, i)
    sns.countplot(x=col, data=customer, order=customer[col].value_counts().index)
    plt.title(f'Distribution of {col}')
    plt.xlabel("")
    plt.ylabel("")
plt.tight_layout()
plt.show()


From these visualizations, we observe that the dataset shows no meaningful variation in gender or location type, suggesting that these variables may have limited discriminative power for segmentation.
Regarding education, the majority of loyalty members hold a Bachelor’s degree, followed by those with College-level education.
In terms of marital status, most customers are married, which may indicate a stable and mature demographic segment.
The loyalty status distribution reveals that most clients belong to the Star tier, followed by Nova and Aurora, highlighting a concentration in mid-level membership.
Finally, the enrollment type indicates that the majority of customers joined through the Standard enrollment process, which could suggest limited participation in promotional or referral programs.


#### 4.1.2 Distribution of Numerical Features

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.countplot(x=customer["EnrollmentDateOpening"].dt.year, data=customer)
plt.title("Enrollment per year")
plt.xlabel("")

plt.subplot(1, 2, 2)
sns.countplot(x=customer["EnrollmentDateOpening"].dt.month, data=customer)
plt.title("Enrollment per month")
plt.xlabel("")
plt.show()
customer.head(3)

The analysis of enrollment trends shows a stable customer acquisition rate between 2016 and 2020, followed by a significant surge in 2021, which represents the highest volume of new enrollments within the dataset. This latest growth suggests successful recent marketing or promotional efforts and indicates a healthy expansion of the customer base. 2021 Promotion

In [ ]:
monthly_enrollments = customer.groupby(customer["EnrollmentDateOpening"].dt.to_period("M")).size().sort_index()
# converter índice PeriodIndex para timestamp para plot ficar com labels de datas
monthly_enrollments.index = monthly_enrollments.index.to_timestamp()

plt.figure(figsize=(14, 6))
monthly_enrollments.plot(kind='line', marker='o', color='teal')
plt.title("Monthly evolution of enrollment")
plt.xlabel("")
plt.ylabel("New clients")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:

valid_dates = customer["CancellationDate"].dt.year != 1904


plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.countplot(x=customer.loc[valid_dates, "CancellationDate"].dt.year.astype(int).sort_values())
plt.title("Cancellations per year")
plt.xlabel("")

plt.subplot(1,2,2)
sns.countplot(x=customer.loc[valid_dates, "CancellationDate"].dt.month.astype(int))
plt.title("Cancellations per month")
plt.xlabel("Month")
plt.ylabel("")
plt.tight_layout()

In [ ]:
monthly_enroll = customer.groupby(customer["EnrollmentDateOpening"].dt.to_period("M")).size()
monthly_cancel = customer.loc[valid_dates].groupby(customer.loc[valid_dates, "CancellationDate"].dt.to_period("M")).size()

# Criar range completo de meses para alinhar as séries
all_months = pd.period_range(
    start=min(monthly_enroll.index.min(), monthly_cancel.index.min()),
    end=max(monthly_enroll.index.max(), monthly_cancel.index.max()),
    freq="M"
)

# Reindexar para garantir continuidade
monthly_enroll = monthly_enroll.reindex(all_months, fill_value=0)
monthly_cancel = monthly_cancel.reindex(all_months, fill_value=0)

# Converter PeriodIndex em datetime (para eixo legível)
monthly_enroll.index = monthly_enroll.index.to_timestamp()
monthly_cancel.index = monthly_cancel.index.to_timestamp()

plt.figure(figsize=(14,6))
plt.plot(monthly_enroll.index, monthly_enroll, marker='o', label="Enrollment", color='teal')
plt.plot(monthly_cancel.index, monthly_cancel, marker='o', label="Cancelation", color='salmon')
plt.title("Enrollments Vs Cancellations", fontsize=14)
plt.xlabel("")
plt.ylabel("Clients")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
numeric_customer = ["Income", "Customer Lifetime Value"]
customer[numeric_customer].hist(bins=30 ,figsize=(10,4))
plt.suptitle("Numeric Feature Distributions")
plt.show()


The distributions of Income and Customer Lifetime Value are highly right-skewed, with the vast majority of customers concentrated at low values. The presence of a long tail suggests potential high-value customers, but these instances should be investigated to determine if they represent a genuine business segment (e.g., corporate clients) or data outliers.


In [ ]:
plt.figure(figsize=(20, 10))
for i, cat_col in enumerate(categorical_customer, 1):
    plt.subplot(2, 4, i)
    sns.boxplot(x=cat_col, y="Customer Lifetime Value", data=customer)
    plt.title(f"CLV by {cat_col}")
    plt.xlabel("")
    plt.ylabel("")
    plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

plt.figure(figsize=(20, 10))
for i, cat_col in enumerate(categorical_customer, 1):
    plt.subplot(2, 4, i)
    sns.boxplot(x=cat_col, y="Income", data=customer)
    plt.title(f"Income by {cat_col}")
    plt.xlabel("")
    plt.ylabel("")
    plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
def find_group_outliers(df, group_col, value_col, k=1.5):
    outlier_idx = []
    for name, g in df.groupby(group_col):
        q1 = g[value_col].quantile(0.25)
        q3 = g[value_col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - k * iqr
        upper = q3 + k * iqr
        mask = (g[value_col] < lower) | (g[value_col] > upper)
        outlier_idx.extend(g[mask].index.tolist())
    return df.loc[outlier_idx]

def outliers_for_all(df, cat_cols, value_cols=("Customer Lifetime Value", "Income"), k=1.5):
    results = {}
    for val in value_cols:
        print(f"\n--- {val} ---")
        # compute outliers per categorical column
        for col in cat_cols:
            out = find_group_outliers(df, col, val, k=k)
            results[(col, val)] = out
            print(f"{col}: {len(out)} outliers")
    return results

outliers_dict = outliers_for_all(customer, categorical_customer)


**Income Distribution by Loyalty Status**

The median Income is similar but slightly lower for the Star loyalty group compared to Aurora and Nova. However, the overall distribution and range of Income across all three groups are highly consistent, with the Interquartile Ranges (IQR) and whiskers spanning similar values (from approximately $0 to $100,000). This suggests that Loyalty Status is not a strong differentiator of customer income levels.

**Customer liftime value by Loyalty Status**

The plot shows that high-CLV customers (outliers) are not segregated within a single, premium loyalty status; they are present across all three loyalty statuses (Star, Aurora, and Nova). While the Aurora tier shows a slightly higher median CLV, the overall pattern suggests that Loyalty Status is a poor predictor of a customer's lifetime value.

**Income Distribution by Education Level**
The analysis reveals that a Bachelor's degree is the dominant factor determining high customer income, with this group showing a significantly higher median and broader Interquartile Range (IQR) than all others. In contrast, Master's, High School, and Doctor levels show consistently low and similar median incomes. Crucially, the College category displays a median income of zero, pointing to a severe data quality issue (missing or improperly coded income values) that must be addressed prior to any predictive modeling.

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Income', y='Customer Lifetime Value', data=customer)
plt.title('Income vs Customer Lifetime Value')
plt.show()

The scatter plot confirms that there is no strong linear correlation between Income and Customer Lifetime Value (CLV). High-CLV customers (the upper outliers) are observed across the entire income spectrum, demonstrating that a customer's purchasing power is not the primary driver of their long-term value. This suggests that predictive models should prioritize behavioral features over income.

### 4.2 Flights Database

In [ ]:
flight.describe().T

#### 4.2.1 Distribution of Numerical Features


In [ ]:
numeric_flights = ["NumFlights", "NumFlightsWithCompanions", "DistanceKM", "PointsAccumulated", "PointsRedeemed", "DollarCostPointsRedeemed"]

# Histograms
flight[numeric_flights].hist(bins=30, figsize=(20, 10))
plt.suptitle("Numeric Feature Distributions")
plt.tight_layout()
plt.show()

# Identificar outliers usando IQR
Q1 = flight[numeric_flights].quantile(0.25)
Q3 = flight[numeric_flights].quantile(0.75)
IQR = Q3 - Q1

outliers = ((flight[numeric_flights] < (Q1 - 1.5 * IQR)) | (flight[numeric_flights] > (Q3 + 1.5 * IQR))).sum()
print("Número de outliers por variável:")
print(outliers)

### 4.3 Multivariate analysis


In [ ]:
flight.groupby(["Year", "Month"])["NumFlights"].sum()

In [ ]:
# Choosing the style 
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(14, 7))

# Create the graph
ax = sns.lineplot(
        data=flight,
        x="Month",
        y="NumFlights",
        hue="Year",
        palette="viridis",  # Color scheme
        linewidth=2.5,
        marker='o',         # Add markers
        markersize=8     
    )

plt.title("Monthly variation in flights per year",
            fontsize=16,
            fontweight='bold',
            pad=20)
plt.ylabel('Number of Flights (10^3)', fontsize=12, labelpad=10)

plt.legend(
        title='Year',
        title_fontsize=12,
        fontsize=10,
        frameon=True,
        fancybox=True,
        framealpha=0.9,
        shadow=True,
        borderpad=1
)

plt.xticks(range(1, 13),
           ['Jan', 'Fev', 'Mar', 'April', 'May', 'Jun',
            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
           rotation=45)
plt.ylim(0)

plt.grid(True, linestyle='--', alpha=0.7)

sns.despine(trim=True, left=True)

plt.tight_layout()
plt.show()

In [ ]:
#sns.pairplot(data=flight, vars= numeric_flights)

#plt.show()

# meti # para nao demorar a correr

During the exploration, several features were found to have strong linear relationships. These are not coincidental correlations but rather indicators of specific business rules used by the airline.

A strong positive correlation was observed between DistanceKM and PointsAccumulated. This is expected, as it reflects the loyalty program's core mechanic: customers earn points based on the distance they fly.

However, the analysis revealed a more precise calculation. The data shows that the number of points awarded is based on a fixed ratio. For example, flights of 2,388 km and 2,380 km both yield 238 points. This strongly suggests that the business rule is PointsAccumulated = floor(DistanceKM / 10).

This means that points are awarded for every full 10 kilometers flown, and any remaining unit-level distance is truncated. Understanding this specific rule is key, as it explains the exact, predictable nature of how points are earned.

Similarly, the strong correlation between PointsRedeemed and DollarCostPointsRedeemed points to a derived relationship, not a behavioral one.

The DollarCostPointsRedeemed is likely an internal accounting metric. It is almost certainly calculated by multiplying the PointsRedeemed by a fixed monetary rate (a "cost-per-point") that the airline uses to track the financial liability of its loyalty program. This makes one variable a direct linear function of the other, explaining the perfect or near-perfect correlation.

The heatmap below comprove the things that we observed

In [ ]:

correlation_matrix = flight[numeric_flights].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, 
            annot=True,           
            cmap='coolwarm',     
            vmin=-1, vmax=1,      
            center=0,             
            square=True,          
            fmt='.2f',            
            cbar_kws={'shrink': 0.8})  

plt.title("Correlation Heatmap - flight")
plt.show()
plt.show()

In [ ]:
numeric_cols = [
    'NumFlights',
    'DistanceKM',
    'PointsAccumulated',
    'PointsRedeemed',
    'DollarCostPointsRedeemed',
    'Customer Lifetime Value',
    'Income'
]

merged = pd.merge(customer, flight, on='Loyalty#', how='inner')
agg = merged.groupby('Loyalty#', as_index=False).agg({
    'NumFlights': 'sum',
    'DistanceKM': 'sum',
    'PointsAccumulated': 'sum',
    'PointsRedeemed': 'sum',
    'DollarCostPointsRedeemed': 'sum',
    'Customer Lifetime Value': 'mean',
    'Income': 'first',
    'LoyaltyStatus': 'first',
    'Gender': 'first',
    'Education': 'first',
})

In [ ]:
#n_rows = len(categorical_customer)
#n_cols = len(numeric_flights)

#fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 4*n_rows))

#for i, cat in enumerate(categorical_customer):
    #for j, num in enumerate(numeric_flights):
        #ax = axes[i, j]
        #sns.boxplot(x=cat, y=num, data=merged, ax=ax)
        #ax.set_title(f'{num} by {cat}')
        #ax.set_xlabel('')
        #ax.set_ylabel('')
        #ax.tick_params(axis='x', rotation=45)

#plt.tight_layout()
#plt.show()

# meti # para nao demorar a correr

In [ ]:
#sns.pairplot(merged[numeric_cols], diag_kind='kde', plot_kws={'alpha':0.6})
#plt.suptitle('Pairplot – Customer & Flight Numerical Variables', y=1.02)
#plt.show()

# so coloquei # para nao demorar a correr


## 5. Data Pre-processing

### 5.1 Feature Engineering

This section aggregates flight-related data for each customer based on their unique Loyalty# identifier.
A reusable function (cumulative_customer) is defined to compute the total sum of a selected column (e.g., number of flights, distance traveled, points accumulated, and points redeemed) per customer.
The resulting aggregated datasets are then merged into the main customer DataFrame to enrich it with cumulative statistics for each individual.

#### 5.1.1 Cumulative Customer Totals

Total values of flights, distance, points redeemed/accumulated, and flights with companions for each customer, capturing overall activity.

In [ ]:
total_flights = cumulative_customer("NumFlights",flight)
total_DistanceKM = cumulative_customer("DistanceKM",flight)
total_PointsRedeemed = cumulative_customer("PointsRedeemed",flight)
total_PointsAccumulated = cumulative_customer("PointsAccumulated",flight)
total_NumFlightsWithCompanions = cumulative_customer("NumFlightsWithCompanions",flight)

In [ ]:
features = {
    "Total NumFlights": total_flights['NumFlights'],
    "Total Distance KM": total_DistanceKM['DistanceKM'],
    "Total Points Redeemed": total_PointsRedeemed['PointsRedeemed'],
    "Total Points Accumulated": total_PointsAccumulated['PointsAccumulated'],
    "Total Flights With Companions": total_NumFlightsWithCompanions['NumFlightsWithCompanions']
}

for name, feat in features.items():
    analyze_feature(feat, feature_name=name)

#### 5.1.2 Seasonality Features

Features that capture seasonal patterns in customer flight behavior, showing periods of higher or lower activity throughout the year.

In [ ]:
seasonality_data = calculate_seasonality(flight)


for col in ['Summer_Ratio', 'Fall_Ratio', 'Spring_Ratio', 'Winter_Ratio', 'Seasonality_Index']:
    analyze_feature(seasonality_data[col], feature_name=col)

#### 5.1.3 Trend Feature

Feature representing the trend of the number of flights per customer over time, indicating whether activity is increasing or decreasing.

In [ ]:
trend_data = calculate_trend(flight)

analyze_feature(trend_data['Flight_Trend_Slope'], feature_name='Flight Trend Slope')

#### 5.1.4 Integrating New Features

Merge all newly created features (cumulative totals, seasonality, and trend) into the main customer dataframe.

In [ ]:
new_columns = [
    total_flights,
    total_DistanceKM,
    total_PointsRedeemed,
    total_PointsAccumulated,
    total_NumFlightsWithCompanions,
    seasonality_data,
    trend_data
]


for data in new_columns:
    customer = customer.merge(data, on="Loyalty#", how="left")

#### 5.1.5 Derived Ratios

Features derived from existing metrics, such as the proportion of flights with companions and average distance per flight, providing more granular insights into customer behavior.

In [ ]:
customer = customer.fillna(0)
customer['Companion_Ratio'] = customer['NumFlightsWithCompanions'] / customer['NumFlights']
customer['Avg_Distance_Flight'] = customer['DistanceKM'] / customer['NumFlights']


customer = customer.replace([np.inf, -np.inf], 0)
customer[['Companion_Ratio', 'Avg_Distance_Flight']] = customer[['Companion_Ratio', 'Avg_Distance_Flight']].fillna(0)

In [ ]:
analyze_feature(customer['Companion_Ratio'], feature_name='Companion Ratio')
analyze_feature(customer['Avg_Distance_Flight'], feature_name='Avg Distance Flight')

#### 5.1.6 Temporal Customer Features

- **Recency**: Months since the last flight.
- **Tenure**: Days since the customer account was client of the company.

In [ ]:
last_flight = (
    flight[flight['NumFlights'] > 0]
    .groupby('Loyalty#')['YearMonthDate']
    .max()
    .reset_index()
    .rename(columns={'YearMonthDate': 'last_flight_date'})
)
reference_date = flight['YearMonthDate'].max()

last_flight['recency_months'] = (
        (reference_date.year - last_flight['last_flight_date'].dt.year) * 12 +
        (reference_date.month - last_flight['last_flight_date'].dt.month)
)
customer = customer.merge(last_flight[['Loyalty#', 'recency_months']], on='Loyalty#', how='left')
customer.replace({"recency_months": {np.nan: -1}}, inplace=True)
customer['recency_months'] = customer['recency_months'].astype(int)

In [ ]:
customer["Tenure"] = (pd.Timestamp.today() - customer["EnrollmentDateOpening"]).dt.days
customer.sort_values(by='Tenure', ascending=True, inplace=True)

In [ ]:
analyze_feature(customer['recency_months'], feature_name='Recency Months')
analyze_feature(customer['Tenure'], feature_name='Tenure')

#### 5.1.7 Engagement/Loyalty Metrics

Proportion of points redeemed vs accumulated, indicating customer engagement with the loyalty program:

- High Value (Near 1): Client that use points frequently, can indicate that the client engage and likes rewards
- Baixo valor (Near 0): Client that don't accumulate, can show some disinterest, or lack of importance/attention with the rewards.


In [ ]:
customer["PointsUtilizationRate"] = customer["PointsRedeemed"] / (customer["PointsAccumulated"] + 1)

In [ ]:
analyze_feature(customer['PointsUtilizationRate'], feature_name='Points Utilization Rate')

#### 5.1.8 New Customer Lifetime Value (CLV)

New normalized metric combining number of flights, income, and tenure to estimate the long-term value of each customer.

In [ ]:
scaler = MinMaxScaler()

cols_to_combine = ['NumFlights', 'Income', 'Tenure']

df_temp_calc = pd.DataFrame(scaler.fit_transform(customer[cols_to_combine]), 
                            columns=cols_to_combine, 
                            index=customer.index)

# nova fórmula de CLV com pesos ajustados
customer['CLV'] = (
    (df_temp_calc['NumFlights'] * 0.50) + 
    (df_temp_calc['Income'] * 0.30) + 
    (df_temp_calc['Tenure'] * 0.20)
)

In [ ]:
analyze_feature(customer['CLV'], feature_name='CLV')

#### (Bonus) Map Exploration

NOTE: 
 The map is interactive. Hover over the data points to view detailed information

In [ ]:
# Group by Province/State and count the number of customers
customermap = customer.groupby("Province or State").agg({
    "Loyalty#": "count",  # Count number of customers
    "Latitude": "first",
    "Longitude": "first",
    "LoyaltyStatus": lambda x: x.mode()[0],
    "NumFlights": "mean",
    "DistanceKM": "mean",
}).reset_index().rename(columns={"Loyalty#": "NumCustomers"})

# Create the map
fig = px.scatter_mapbox(
    customermap,
    lat="Latitude",
    lon="Longitude",
    size="NumCustomers",
    color="NumCustomers",
    hover_name="Province or State",
    hover_data={"NumCustomers": True, "Latitude": False, "Longitude": False, "NumFlights": True, "DistanceKM" : True, "LoyaltyStatus": True},
    zoom=3,
    height=700,
    title="Number of Customers by Canadian Province/State",
    color_continuous_scale=px.colors.sequential.Viridis
)

# Update layout for better visualization
fig.update_layout(
    mapbox_style="carto-positron",
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
    coloraxis_colorbar=dict(
        title="Number of Customers",
        thicknessmode="pixels",
        thickness=20,
        lenmode="pixels",
        len=300,
        yanchor="top",
        y=1,
        xanchor="left",
        x=0.01
    )
)

fig.show()

### 5.2 Feature Encoding

In [ ]:
customer.columns

In [ ]:
metric_features = [
    "Income",
    "CLV",
    "NumFlights",
    "DistanceKM",
    "PointsRedeemed",
    "PointsAccumulated",
    "NumFlightsWithCompanions",

    "Fall_Ratio",
    "Spring_Ratio",
    "Summer_Ratio",
    "Winter_Ratio",
    "Seasonality_Index",

    "Flight_Trend_Slope",
    "Companion_Ratio",
    "Avg_Distance_Flight",

    "recency_months",
    "Tenure",
    "PointsUtilizationRate"
]

categorical_features = [
    'Gender',
    'Education',
    'Marital Status',
    'EnrollmentType',
    'Province or State',
    'City'
]

In [ ]:
customer_oh = pd.get_dummies(customer[categorical_features], prefix='oh', drop_first=True, dtype=int)
customer = pd.concat([customer, customer_oh], axis=1)
non_metric_features = customer.columns[customer.columns.str.startswith('oh_')].tolist()

In [ ]:
features_to_use = metric_features + non_metric_features

unused_features = [col for col in customer.columns if col not in features_to_use]

print(f"Features Métricas ({len(metric_features)}): {metric_features}")
print(f"Features One-Hot ({len(non_metric_features)}): {non_metric_features}")
print(f"Features Unused/Descartadas ({len(unused_features)}): {unused_features}")

In [ ]:
customer = customer[metric_features + non_metric_features].copy()

### 5.3 Feature Selection

In [ ]:
columns_to_drop = [
    # --- Identificadores & PII ---
    'Loyalty#', 'First Name', 'Last Name', 'Customer Name',
    'Country', 'Postal code', 'Latitude', 'Longitude',

    # --- Originais categóricas (já codificadas) ---
    'Gender',
    'Education',
    'Marital Status',
    'EnrollmentType',
    'Province or State',
    'City',
    'Location Code',
    'LoyaltyStatus',

    # --- Datas ---
    'EnrollmentDateOpening',
    'CancellationDate',

    # --- Redundâncias ---
    'Customer Lifetime Value',
    'DistanceKM',
    'PointsRedeemed',
    'PointsAccumulated',
    'NumFlightsWithCompanions'
]

customer = customer.drop(columns=columns_to_drop, errors='ignore')
customer.head(3)

fs_metric_features = [f for f in metric_features if f in customer.columns]

In [ ]:
plt.figure(figsize=(9,7))
corr = customer[fs_metric_features].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap – Customer & Flight Variables')
plt.show()

### 5.4 Feature Scalling

#### 5.4.1 MinMaxScaler

In [ ]:
df_minmax = customer.copy()

In [ ]:
mm_scaler = MinMaxScaler()
mm_scaled_feat = mm_scaler.fit_transform(df_minmax[fs_metric_features])

df_minmax[fs_metric_features] = mm_scaled_feat
df_minmax[fs_metric_features].describe().T.round(2)

MinMaxScaler compresses all data points into a fixed range between 0 and 1, where the lowest value becomes 0 and the highest becomes 1, ensuring that no single variable dominates the model simply because it has larger raw numbers.

#### 5.4.2 StandardScaler

In [ ]:
df_standard = customer.copy()

In [ ]:
scaler = StandardScaler()
ss_scaled_feat = scaler.fit_transform(customer[fs_metric_features])

df_standard[fs_metric_features] = ss_scaled_feat
df_standard[fs_metric_features].describe().T.round(2)

StandardScaler transforms the data so that the mean is 0 and the standard deviation is 1. It tells us how many standard deviations a data point is above or below the average, without imposing fixed maximum or minimum limits.

#### 5.4.3 Comparing different scalers vs original data

In [ ]:

sns.set_style('whitegrid')
# Aumentar o figsize para que 6 boxplots fiquem legíveis
fig, axes = plt.subplots(2, 3, figsize=(8, 5), tight_layout=True, sharey='row')

bp_feat_l = 'Income'

sns.boxplot(customer, x=bp_feat_l, ax=axes[0][0], width=.4)
axes[0][0].set_title('Original')
axes[0][0].set_ylabel(bp_feat_l)

sns.boxplot(df_minmax, x=bp_feat_l, ax=axes[0][1], width=.4)
axes[0][1].set_title('MinMaxScaler()')

sns.boxplot(df_standard, x=bp_feat_l, ax=axes[0][2], width=.4)
axes[0][2].set_title('StandardScaler()')



bp_feat_r = 'NumFlights'

sns.boxplot(customer, x=bp_feat_r, ax=axes[1][0], width=.4)
axes[1][0].set_ylabel(bp_feat_r)

sns.boxplot(df_minmax, x=bp_feat_r, ax=axes[1][1], width=.4)

sns.boxplot(df_standard, x=bp_feat_r, ax=axes[1][2], width=.4)


axes[1][0].set_xlabel(None)
axes[1][1].set_xlabel(None)
axes[1][2].set_xlabel(None)

fig.suptitle('Boxplots: "{}" and "{}"'.format(bp_feat_l, bp_feat_r))
plt.show()

In [ ]:
sns.set_style('white')

fig, axes = plt.subplots(3,1, figsize=(8,5), tight_layout=True,)

sns.boxplot(customer, x=bp_feat_l, ax=axes[2], width=.4)
sns.boxplot(df_minmax, x=bp_feat_l, ax=axes[1], width=.4)
sns.boxplot(df_standard, x=bp_feat_l, ax=axes[0], width=.4)

axes[0].set_title("StandardScaler()")
axes[0].set_xlabel(None)
axes[1].set_title("MinMaxScaler()")
axes[1].set_xlabel(None)
axes[2].set_title("original")
axes[2].set_xlabel(None)

plt.suptitle('Boxplots: "{}"'.format(bp_feat_l))

As we can see through the boxplots, MinMaxScaler forces the data into the 0-1 interval destroying the subtle differences in variance among average customers. This compression leads to clustering results that are less discriminatory.

StandardScaler preserves the relative distances between data points. This makes it ideal for the distance-based algorithms that we will use later, as it ensures that distances are calculated fairly and robustly across all features.

In [ ]:
sns.set_style('whitegrid')

# Create subplots with 3 rows, 1 column:
fig, axes = plt.subplots(3,1, 
                         figsize=(5,5), tight_layout=True)

# Which feature do you want to visualize?
hp_feat = 'Income'

# common function args for histplot
hp_args = dict(x=hp_feat, bins=15)


sns.histplot(customer, ax=axes[0], **hp_args)
axes[0].set_title('{}: Original'.format(hp_feat))
axes[0].set_xlabel(None)

sns.histplot(df_minmax, ax=axes[1], **hp_args)
axes[1].set_title('{}: MinMaxScaler()'.format(hp_feat))
axes[1].set_xlabel(None)

sns.histplot(df_standard, ax=axes[2], **hp_args)
axes[2].set_title('{}: StandardScaler()'.format(hp_feat))
axes[2].set_xlabel(None)


plt.show()

sns.set()

We decided to keep the StandardScaler because it best preserved relative distances across features and provided the most suitable scaling for our clustering objectives.

In [ ]:
customer = df_standard.copy()

In [ ]:
sns.set()

### 5.5 Outliers removal (DBSCAN)

We create a clean dataset using DBSCAN for K-Means training, ensuring that centroids are not distorted by extreme values. However, these outliers are saved in df_outliers for separate analysis, as they can also represent valuable customers.

In [ ]:
len(fs_metric_features)

In [ ]:
neigh = NearestNeighbors(n_neighbors=28) #2* nr de features
neigh.fit(customer[fs_metric_features])
distances, _ = neigh.kneighbors(customer[fs_metric_features])

# Ordenar e plotar
distances = np.sort(distances[:, -1], axis=0)

plt.figure(figsize=(10, 6))
plt.plot(distances)
plt.title('K-Distance Graph for eps Selection', fontsize=14)
plt.xlabel('customers')
plt.ylabel('Epsilon')
plt.grid(True, alpha=0.3)
plt.axhline(y=1.8, color='r', linestyle='--', label='Potential eps (elbow)')
plt.ylim(0,4)
plt.show()

In [ ]:
all_customers = customer.copy()

In [ ]:
dbscan = DBSCAN(eps=1.8, min_samples=28, n_jobs=-1) # eps was later found using the K-Distance Graph for eps Selection
dbscan_labels = dbscan.fit_predict(customer[fs_metric_features]) # ou features_modelo

outlier_count = Counter(dbscan_labels)
print(f"DBSCAN results: {outlier_count}")
print(f"Outliers detected (-1): {outlier_count[-1]}")
n_core = sum([count for label, count in outlier_count.items() if label != -1])
print(f"Core customers: {n_core}")

customer_out = customer[dbscan_labels == -1].copy()

# data frame limpo scaled para o kmeans
customer = customer[dbscan_labels != -1].copy()

print(f"\nWorking with {len(customer):,} core customers, {np.round(customer.shape[0] / all_customers.shape[0], 4)}")
print(f"Outliers saved for later: {len(customer_out)}")

## 6. Data Segmentation

### 6.1 RFM Analysis

alterar M depois do feature engineering

To better understand customer behavior and identify meaningful subgroups, we applied an RFM (recency, frequency, monetary) segmentation model. This method classifies customers based on:
- Recency (R): how recently they flew;
- Frequency (F): how often they fly;
- Monetary (M): how much money they spend.

This allows us to highlight high-value customers, at-risk customers, and behavioral patterns relevant for targeted marketing strategies.

atencao: alterar depois monetary (atualmente é CLV, mas temos de mudar)

In [ ]:
def create_rfm_quantiles(df, recency, frequency, monetary, rec_ascending=True, freq_ascending=False, mon_ascending=False):
    rfm_df = df.copy()

    rfm_df["R_quintile"] = pd.qcut(rfm_df[recency].rank(method="first", ascending = rec_ascending),
                                   q=5, labels=[1,2,3,4,5]
                                   ).astype(int)
    rfm_df["F_quintile"] = pd.qcut(rfm_df[frequency].rank(method="first", ascending = freq_ascending),
                                   q=5, labels=[1,2,3,4,5]
                                   ).astype(int)
    rfm_df["M_quintile"] = pd.qcut(rfm_df[monetary].rank(method="first", ascending = mon_ascending),
                                   q=5, labels=[1,2,3,4,5]
                                   ).astype(int)

    rfm_df["RFM_score"] = (rfm_df["R_quintile"].astype(str) + rfm_df["F_quintile"].astype(str) + rfm_df["M_quintile"].astype(str))

    return rfm_df


In [ ]:
customer = create_rfm_quantiles(customer,recency="recency_months",frequency="NumFlights", monetary="CLV", rec_ascending=False, freq_ascending=True, mon_ascending=True)
customer.head()
customer.iloc[:,17:]

Customer segmentation

We defined a set of business-driven RFM rules to map each customer into a meaningful segment.  
This includes groups such as:

| Segment                |  RFM  | Description                                                                 |  Marketing                                              |
|------------------------|-------------------|-----------------------------------------------------------------------------|-----------------------------------------------------------------------------|
| **High-value Loyal customers**          | 555               | Recently active, purchase frequently, and spend the most.                   | VIP programs, exclusive rewards, early access to new products.             |
| **Loyal Frequent customers**    | X5X               | High purchase frequency, even if not always recent.                         | Personalized offers, loyalty nurturing, regular engagement.                |
| **Big spenders**       | XX5               | Highest monetary contribution but not necessarily frequent or recent.       | Promote premium products, upselling opportunities.                         |
| **Potential Loyalists**| 515 / 525         | Recent high spenders with still low frequency.                               | Encourage repeat purchases with progressive discounts or subscriptions.    |
| **New customers**      | 511 / 512         | Recently acquired but with low spending and low frequency.                  | Welcome campaigns, onboarding flows, recurring purchase incentives.        |
| **At Risk customers**  | 155 / 255         | Previously high frequency and/or high spenders, but not recent.             | Re-engagement campaigns, special comeback offers.                          |
| **Lost customers**     | 155  | High-value customers who have been inactive for a long time.                | Aggressive recovery campaigns, personalized incentives.                    |
| **Lost Cheap customers** | 111             | Long inactive, low frequency, and low spending.                             | Avoid spending resources trying to reacquire.                              |
| **Others**             | Mixed             | Customers who don’t fit cleanly into the above priority groups.             | General marketing strategies depending on overall business goals.          |


In [ ]:
def assign_segment(rfm):
        r, f, m = int(rfm[0]), int(rfm[1]), int(rfm[2])

        if r == 5 and f == 5 and m == 5:
            return "High-value Loyal customers"

        if f == 5 and not (r == 5 and m == 5):  
            return "Loyal Frequent customers"

        if m == 5 and not (r == 5 and f == 5):
            return "Big spenders"

        if r == 5 and m == 5 and f in [1, 2]:
            return "Potential Loyalists"

        if r == 5 and f == 1 and m in [1,2]:
            return "New customers"

        if (r in [1,2]) and f == 5 and m == 5:
            return "At Risk customers"

        if r == 1 and f == 5 and m == 5:
            return "Lost customers"

        if r == 1 and f == 1 and m == 1:
            return "Lost Cheap customers"

        return "Others"


In [ ]:
customer["RFM_segment"] = customer["RFM_score"].apply(assign_segment)
customer["RFM_segment"].value_counts().plot(kind="bar")
customer["RFM_segment"].value_counts()

The final distribution of segments highlights the behavioral diversity within the customer base.  
A larger presence of groups such as Loyal customers or Big spenders indicates strong retention and high engagement.  
Conversely, the proportion of At Risk or Lost customers reveals potential churn issues that may require dedicated retention strategies.

## 7. Data Clustering

In [ ]:
fs_metric_features

**Perspectives**

In [ ]:
clustering_perspectives = {
    # Value-based segmentation: assesses the economic value and long-term
    # relationship of each customer with the company.
    "Value_Segmentation": [
        'CLV',
        'Income',
        'PointsUtilizationRate',
        'Tenure'
    ],

    # Behavioral segmentation: analyzes how customers travel and how
    # their travel behavior evolves over time.
    "Behavioral_Segmentation": [
        'NumFlights',
        'Avg_Distance_Flight',
        'recency_months',
        'Companion_Ratio',
        'Flight_Trend_Slope'
    ],

    # Seasonality segmentation: captures when customers travel during
    # the year and their dependency on specific seasons.
    "Seasonality_Segmentation": [
        'Fall_Ratio',
        'Spring_Ratio',
        'Summer_Ratio',
        'Winter_Ratio',
        'Seasonality_Index'
    ]
}

In [ ]:
# Demographic segmentation: quem são os clientes em termos de perfil socioeconómico?
Demographic_Segmentation = [
        'Province or State',
        'City',
        'Location Code',
        'Gender',
        'Education',
        'Marital Status'
]

### 7.1 Value segmentation perspective

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
features_val = clustering_perspectives['Value_Segmentation']

In [ ]:
results_value = []

#### 7.1.1 K-means and Hierarchical clustering

To ensure a robust segmentation, we will not rely solely on K-Means.
We will compare it against Hierarchical Clustering to validate the number of clusters k and check if the customer groups have irregular shapes. The algorithm that best explains the variance R^2 of the data will be selected for the final profiling.

In [ ]:
df_value_kmeans_hc = customer[features_val].copy()
df_value_kmeans_hc

In [ ]:
kmeans = KMeans(
    init='k-means++',
    n_init=20,
    random_state=42
)

In [ ]:
hierarchical = AgglomerativeClustering(
    metric='euclidean'
)

In [ ]:
df_value_sample = df_value_kmeans_hc.sample(n=5000, random_state=42)

We used a subset of the data to determine the optimal number of clusters k and the best clustering approach, overcoming the constraints about calculating the distance matrix for the entire population.

In [ ]:
os.environ["OMP_NUM_THREADS"] = "1"

r2_scores = {}

r2_scores['kmeans'] = func.get_r2_scores(df_value_sample, df_value_kmeans_hc.columns.tolist(), kmeans)

for linkage in ['complete', 'average', 'single', 'ward']: # 4 different versions of HC
    r2_scores[linkage] = func.get_r2_scores(
        df_value_sample,                 # data
        df_value_kmeans_hc.columns.tolist(),   # features of perspective
        # use HClust, changing the linkage at each iteration
        hierarchical.set_params(linkage=linkage) 
    )

In [ ]:
r2_scores_df = pd.DataFrame(r2_scores)
r2_scores_df

In [ ]:
r2_scores_df.plot.line(figsize=(10,7))

plt.title("Demographic Variables:\nR² plot for various clustering methods\n", fontsize=21)
plt.legend(title="Cluster methods", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13)
plt.show()

The comparison identifies K-Means as the best algorithm for this dataset, since it achieves the highest R^2 scores across all cluster counts.
Analyzing the elbow of the curve reveals that 4 clusters is the optimal solution.

In [ ]:
df_sample_value_viz = df_value_kmeans_hc.sample(n=3000, random_state=42)

To optimize processing time, we will generate the Silhouette plot using a random subset of 3,000 customers.

In [ ]:
func.visualize_silhouette_graf(df_sample_value_viz, range_clusters=range(2, 11))

MUDAR Although R^2 plot pointed towards a 4-cluster solution, the analysis of the Silhouette Plot favored k=3.
Since the Silhouette score dropped when moving from 3 to 4 clusters (indicating less separation between groups), we chose the Elbow suggestion. We selected k=3 as the final solution to guarantee the most reliable and distinct separation of customer value tiers.

In [ ]:
kmeans_value = KMeans(
    n_clusters=4,
    init='k-means++',
    n_init=20,
    random_state=42
)

In [ ]:

value_labels = kmeans_value.fit_predict(df_value_kmeans_hc)
value_labels

In [ ]:
print(pd.Series(value_labels).value_counts().sort_index())

#### 7.1.2 DBSCAN

In [ ]:
k_val = plot_k_distance(customer, features_val)

In [ ]:
dbscan_results = {}

labels_val, n_clus_val, dist_val = get_dbscan(
    customer, 
    features_val, 
    eps=0.35,
    min_samples=7
)

dbscan_results['Value'] = {'Clusters': n_clus_val, 'Distribution': dist_val}

Here we can see a good example of a bad clustering solution, where DBSCAN creates a large number of small clusters and a few large clusters. Its does not provide a clear separation between clusters and it is not a good solution for this dataset.

#### 7.1.3 HDBSCAN

In [ ]:
labels_hdb_val, n_hdb_val, dist_hdb_val = func.get_hdbscan(
    customer, 
    features_val, 
    min_cluster_size=200, 
    min_samples=7         
)

Compared to DBSCAN, HDBSCAN creates a more balanced distribution of clusters, with a more uniform size of clusters. But we have 2 groups that are bigger than the others, which is not a good solution for this dataset.

#### 7.1.4 Mean-shift

In [ ]:
df_value_meanshift= customer.copy()
df_value_meanshift['Cluster_MS_val'] = get_meanshift(customer, features_val)

In [ ]:
print(df_value_meanshift.groupby('Cluster_MS_val')[features_val].mean())
print("\nCluster Sizes:")
print(df_value_meanshift.groupby('Cluster_MS_val').size())

#### 7.1.5 Gaussian Mixture Model

In [ ]:
df_value_gaussian_mix_model= customer.copy()

In [ ]:
get_n_components(df_value_gaussian_mix_model, features_val)

The optimal number of components ($K$) for the Gaussian Mixture Model was determined by minimizing the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC). These metrics were evaluated for both Diagonal and Full covariance structures across a range of 1 to 10 components.

The **Full Covariance** structure is strictly superior to the Diagonal structure for this dataset.
* **Magnitude of Error:** The Full Covariance model achieves criterion scores as low as ~25,000. In contrast, the Diagonal model’s best score is significantly higher, being around ~40,000.
* **Implication:** The substantial gap in scores confirms that the features in the dataset are correlated. The Diagonal model's assumption of independence is too restrictive and fails to capture the true shape of the data.

**Component Selection (Full Covariance)**
The Full Covariance plot exhibits a distinct "stepped" pattern rather than a smooth curve, revealing the hierarchical nature of the data:
* **Primary Structure ($K=6$):** A sharp, continuous reduction in AIC/BIC is observed up to 6 components, followed by a distinct **plateau** from $K=6$ to $K=8$. This indicates that 6 clusters are sufficient to capture the coarse, high-level structure of the data.


**Decision:** A **Full Covariance GMM with 6 components** is selected to prioritize strategic applicability and model parsimony.

In [ ]:
gmm_model_value, df_concat_value = fit_gmm_segmentation(df_value_gaussian_mix_model, features_val, k=7, rsq_func=get_rsq)

In [ ]:
analyze_gmm_uncertainty(gmm_model_value, df_value_gaussian_mix_model, features_val, threshold=0.8)

**Uncertainty Analysis**
* **Threshold:** A strict confidence threshold of **80%** was applied to test the robustness of the clusters.
* **Results:** The model successfully classified **83.6%** of customers (12,533) with high confidence. The remaining **16.4%** (2,456 customers) lie in the "boundary" regions between clusters.

#### 7.1.6 SOM

##### 7.1.6.1 Component planes and U-matrix

In [ ]:
grid_size = 12
features_to_use = features_val

# Inicializar e Treinar
som_val = MiniSom(x=grid_size, y=grid_size, input_len=len(features_to_use),
                  topology='hexagonal', sigma=1.0, learning_rate=0.5,
                  random_seed=42)

som_val.pca_weights_init(customer[features_to_use].values)
som_val.train_batch(customer[features_to_use].values, 20000, verbose=True)

weights_val = som_val.get_weights()


In [ ]:
"""
Sugestão Bruna Ver
"""

X = customer[features_to_use].values.astype(np.float32)

som_val = MiniSom(
    x=grid_size, y=grid_size,
    input_len=X.shape[1],
    topology='hexagonal',
    sigma=1.0,
    learning_rate=0.5,
    random_seed=42
)

som_val.random_weights_init(X)
som_val.train_random(X, 3000, verbose=False)

weights_val = som_val.get_weights()

In [ ]:
n_plots = len(features_to_use)
n_cols = 2 
n_rows = math.ceil(n_plots / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 8))
axes = axes.flatten()

for i, feature_name in enumerate(features_to_use):
    # Extrair os pesos desta variável
    feature_weights = weights_val[:, :, i]
    
    # Desenhar o mapa (assumindo que já tens a função plot_hexagons carregada)
    plot_hexagons(feature_weights, som_val, axes[i], label=feature_name, cmap=cm.Spectral_r)

plt.suptitle("Component Planes: Value Perspective", fontsize=16)
plt.tight_layout()
plt.show()

**Income & CLV**: The Income and CLV maps display a near-perfect visual match, with identical "hotspots" in the same regions. This confirms a strong positive correlation: our most affluent customers are directly translating their purchasing power into long-term value.

**Tenure**: The Tenure map is predominantly red, indicating that the vast majority of the customer base consists of long-term loyalists. However, the distinct blue zones in the top-left and bottom-right corners represent recent acquisitions (New Customers). Crucially, these "blue" areas overlap with lower-income zones on the other maps, suggesting that new customers are entering at lower value tiers compared to the established "VIP" veterans.

 **Points Utilization Rate**: The Points Utilization Rate map shows a distinct pattern, mostly blue/green with isolated red pockets that do not perfectly align with the highest Income areas. This indicates that points engagement is a specific behavior driven by a "Smart Redeemer" niche, rather than being a universal trait of wealthy customers.


In [ ]:
plot_som_diagnostics(som_val, customer[features_val].values)

The diagnostic plots reveal a strictly polarized customer base. The Hits Map identifies two major density centers: a massive cluster in the top-left (correlated with Mass Market/Lower Value profiles) and a secondary, dense cluster in the bottom-right (High Value).

Crucially, the U-Matrix shows clear high-distance barriers (red zones) surrounding the bottom-right cluster. This confirms that our High-Value customers form a distinct 'island' with unique characteristics, completely separate from the general customer population. This separation justifies a dedicated, exclusive marketing strategy for this group, as they do not share behaviors with the rest of the database.

##### 7.1.6.2 SOMS with K-means

In [ ]:
labels_som_val = run_som_kmeans(som_val, customer[features_val].values, n_clusters=5) # we tried n_clusters=4 first

Initial validation metrics (Elbow Method, Silhouette Analysis) suggested an optimal structure of k=4. However, visual inspection of the SOM projection highlighted a critical limitation in this configuration: the 'New Customer' cluster was aggregating two financially opposing profiles (High vs. Low Income) solely based on their shared low tenure. Therefore, we opted for a k=5 solution.

The map now clearly delineates 5 core segments:

Yellow (Top-Left): "Low-Value Newcomers." Recent acquisitions with low spending power. 

Orange (Bottom-Right): "High-Potential Newcomers." Recent acquisitions with high income and CLV. 

Burgundy (Bottom-Left): "Loyal Veterans." The stable, long-term high-value core. 

Purple (Top/Center): "Mass Market." High volume but lower individual margins. 

Green (Right/Center): "Transitional." Middle-tier customers showing potential for growth. 


The fact that the clusters appear as solid, contiguous regions (rather than scattered noise) confirms that the K-Means algorithm has identified genuine, well-separated patterns in the customer behavior, rather than forcing arbitrary divisions.

##### 7.1.6.3 SOMS with Hierarchical clustering

In [ ]:
run_som_hierarchical(som_val, n_clusters=5)

The Hierarchical Clustering projection (Ward linkage) produces a topological map nearly identical to the K-Means solution. This convergence between two mathematically distinct algorithms confirms that the 5-cluster structure is stable, natural, and robust. The Dendrogram displays a deep hierarchical split, validating the decision to use k=5. The distinct vertical branches prove that the 5 groups are well-separated in the data space.

The SOM successfully transformed complex multidimensional data into an interpretable 2D map organized by behavioral similarity.

The most significant outcome was the visual diagnosis of the "New Customer" segment. While standard statistics grouped all recent acquisitions together, the SOM geographically separated them into High-Potential and Low-Value zones. This insight was the decisive factor in refining the segmentation from 4 to 5 clusters, allowing for a differentiated retention strategy.

Hits Map demonstrated a healthy data distribution without signs of overfitting. The structural consistency was further validated by the convergence with Hierarchical Clustering results, proving the model is stable and reliable for decision-making.

#### Value segmentation results

In [ ]:
X_value = customer[features_val]
kmeans_final = KMeans(n_clusters=5, init='k-means++', n_init=20, random_state=42)
labels_kmeans = kmeans_final.fit_predict(X_value)

metrics_kmeans = get_model_metrics(
    df=X_value,
    labels=labels_kmeans,
    model_name="K-Means (k=4)",
    perspective="Value"
)
results_value.append(metrics_kmeans)


# --- 2. Hierarchical (Agglomerative) - Ward, k=5 ---

#hc_final = AgglomerativeClustering(n_clusters=5, metric='euclidean', linkage='ward')
#labels_hc = hc_final.fit_predict(X_value)
#metrics_hc = get_model_metrics(
#    df=X_value,
#    labels=labels_hc,
#    model_name="Hierarchical (Ward, k=5)",
#    perspective="Value"
#)
#results_value.append(metrics_hc)


# --- 3. DBSCAN  ---

metrics_dbscan = get_model_metrics(
    df=X_value,
    labels=labels_val, 
    model_name="DBSCAN (eps=0.35)",
    perspective="Value"
)
results_value.append(metrics_dbscan)


# --- 4. HDBSCAN  ---
metrics_hdb = get_model_metrics(
    df=X_value,
    labels=labels_hdb_val,
    model_name="HDBSCAN (min_cluster=200)",
    perspective="Value"
)
results_value.append(metrics_hdb)


# --- 5. Mean Shift ---
metrics_ms = get_model_metrics(
    df=X_value,
    labels=df_value_meanshift['Cluster_MS_val'],
    model_name="Mean Shift",
    perspective="Value"
)
results_value.append(metrics_ms)


# --- 6. Gaussian Mixture (GMM, k=14) ---

labels_value_gmm = gmm_model_value.predict(X_value)

metrics_gmm = get_model_metrics(
    df=X_value,
    labels=labels_value_gmm,
    model_name="GMM (k=14)",
    perspective="Value"
)
results_value.append(metrics_gmm)

In [ ]:
df_compare_value = pd.DataFrame(results_value)
df_compare_value = df_compare_value.sort_values(by='Silhouette Score', ascending=False)
df_compare_value

### 7.2 Behavioral Segmentation perspective

In [ ]:
features_behave = clustering_perspectives['Behavioral_Segmentation']

In [ ]:
results_behave = []

#### 7.2.1 K-means with Hierarchical clustering

In [ ]:
df_behave_kmeans_hc = customer[features_behave].copy()
df_behave_kmeans_hc

In [ ]:
df_behave_sample = df_behave_kmeans_hc.sample(n=5000, random_state=42)

In [ ]:
r2_scores_behave = {}

r2_scores_behave['kmeans'] = func.get_r2_scores(df_behave_sample, df_behave_kmeans_hc.columns.tolist(), kmeans)

for linkage in ['complete', 'average', 'single', 'ward']: # 4 different versions of HC
    r2_scores_behave[linkage] = func.get_r2_scores(
        df_behave_sample,                 # data
        df_behave_kmeans_hc.columns.tolist(),   # features of perspective
        # use HClust, changing the linkage at each iteration
        hierarchical.set_params(linkage=linkage) 
    )

In [ ]:
r2_scores_behave = pd.DataFrame(r2_scores)
r2_scores_behave

In [ ]:
r2_scores_df_behave = pd.DataFrame(r2_scores)

r2_scores_df_behave.plot.line(figsize=(10,7))

plt.title("Demographic Variables:\nR² plot for various clustering methods\n", fontsize=21)
plt.legend(title="Cluster methods", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13)
plt.show()

The comparison of clustering algorithms reveals that K-Means outperforms other methods, achieving the highest explained variance across all k values. Regarding the number of clusters, we identify a elbow at k=4. The model explains mora then 60% of the behavioral variance at this point.

In [ ]:
df_sample_behave_viz = df_behave_kmeans_hc.sample(n=3000, random_state=42)

To optimize processing time, we will generate the Silhouette plot using a random subset of 3,000 customers.

In [ ]:
func.visualize_silhouette_graf(df_sample_behave_viz, range_clusters=range(2, 11))

Although the Silhouette analysis indicates a mathematical peak at k=2, we rejected this solution as it results in an oversimplified binary segmentation ( merely separating high vs. low values). We selected k=5 because it offers the optimal balance between statistical validity and business utility, allowing us to identify distinct niche profiles that would otherwise be merged in a 2-cluster solution.

In [ ]:
kmeans_value = KMeans(
    n_clusters=5,
    init='k-means++',
    n_init=20,
    random_state=42
)

In [ ]:
behave_labels = kmeans_value.fit_predict(df_behave_kmeans_hc)
behave_labels

In [ ]:
print(pd.Series(behave_labels).value_counts().sort_index())

First we tried using k=4, but it didn't show a good distribution of the clusters, so we decided to use k=5 because here we can see more homogenous distribution of the clusters.

#### 7.2.2 DBSCAN

In [ ]:
k_behave = plot_k_distance(customer, features_behave)

In [ ]:
labels_behave, n_clus_behave, dist_behave = get_dbscan(
    customer, 
    features_behave, 
    eps=0.45,
    min_samples=9
)

dbscan_results['Behave'] = {'Clusters': n_clus_behave, 'Distribution': dist_behave}

Once again, DBSCAN didn't show a good distribution of the clusters, so we rejected this solution.

#### 7.2.3 HDBSCAN

In [ ]:
labels_hdb_behave, n_hdb_behave, dist_hdb_behave = func.get_hdbscan(
    customer, 
    features_behave, 
    min_cluster_size=200, 
    min_samples=9         
)

As we can see here the HDBSCAN solution is not a good solution for this dataset, beside the clusters being not homogenous, we also have a 43% of noise points.

#### 7.2.4 Mean-shift

In [ ]:
df_behave_meanshift= customer.copy()
df_behave_meanshift['Cluster_MS_behave'] = get_meanshift(customer, features_behave)

In [ ]:
print(df_behave_meanshift.groupby('Cluster_MS_behave')[features_behave].mean())
print("\nCluster Sizes:")
print(df_behave_meanshift.groupby('Cluster_MS_behave').size())

#### 7.2.5 Gaussian Mixture Model

In [ ]:
df_behave_gaussian_mix_model= customer.copy()


In [ ]:
get_n_components(df_behave_gaussian_mix_model, features_behave)

The **Full Covariance** structure is generally superior to the Diagonal structure for this dataset.
* **Magnitude of Error:** The Full Covariance model achieves criterion scores dropping rapidly to around -65,000 at $K=3$ and reaching lows near -120,000 at $K=10$. The Diagonal model shows a similar trend but generally lags in performance, requiring more components to achieve similar fit levels.
* **Implication:** The better performance of the Full Covariance model confirms that the features in this subset are correlated. The Diagonal model's assumption of independence is too restrictive to efficiently capture the data's shape with a small number of clusters.

**Component Selection (Full Covariance)**
The Full Covariance plot exhibits a very distinct and early **"Elbow" pattern**:
* **Initial Drop ($K=1$ to $K=3$):** A precipitous reduction in AIC/BIC values is observed from 1 to 3 components. The score drops vertically from ~140,000 to roughly -65,000, indicating that the vast majority of the data's structure is captured by just three primary archetypes.
* **The "Elbow" ($K=3$):** The curve flattens noticeably at **3 components**. While the score continues to decrease slowly towards $K=5$ and beyond, the rate of improvement slows down dramatically after the third component.
* **Diminishing Returns:** Beyond $K=3$, the curve transitions into a gentler slope. The massive strategic gain happens in the first three clusters, suggesting that adding further complexity yields diminishing returns relative to the increased model complexity.

**Decision:** A **Full Covariance GMM with 3 components** is selected to prioritize interpretability and capture the core structure of the data without over-fitting.

In [ ]:
gmm_model_behave, df_concat_behave = fit_gmm_segmentation(df_behave_gaussian_mix_model, features_behave, k=3, rsq_func=get_rsq)

In [ ]:
analyze_gmm_uncertainty(gmm_model_behave, df_behave_gaussian_mix_model, features_behave, threshold=0.99)

**Uncertainty Analysis**
* **Threshold:** An extremely strict confidence threshold of **99%** was applied to test the robustness of the clusters.
* **Results:** The model demonstrated nearly perfect separation, as shown by the dense concentration of assignment probabilities above 0.995. The vast majority of customers fall into the "confident" region, with negligible uncertainty found even at this stringent cutoff.


#### 7.2.6 SOM

##### 7.2.6.1 Component planes and U-matrix

In [ ]:
grid_size = 12
features_to_use = features_behave

# Inicializar e Treinar
som_behave = MiniSom(x=grid_size, y=grid_size, input_len=len(features_to_use),
                  topology='hexagonal', sigma=1.0, learning_rate=0.5,
                  random_seed=42)

som_behave.pca_weights_init(customer[features_to_use].values)
som_behave.train_batch(customer[features_to_use].values, 20000, verbose=True)

weights_behave = som_behave.get_weights()

In [ ]:
n_plots = len(features_behave)
n_cols = 3 
n_rows = math.ceil(n_plots / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 9))
axes = axes.flatten()

# 4. Loop through features and plot
for i, feature_name in enumerate(features_behave):
    # Extract weights for this specific feature
    feature_weights = weights_behave[:, :, i]
    
    # Plot the hexagon map
    # Using Spectral_r: Red = High Values, Blue = Low Values
    plot_hexagons(feature_weights, som_behave, axes[i], label=feature_name, cmap=cm.Spectral_r)

# 5. Hide the empty subplot (since we have 5 plots for 6 spaces)
for j in range(n_plots, len(axes)):
    axes[j].axis('off')

plt.suptitle("Component Planes: Behavioral Perspective", fontsize=16)
plt.tight_layout()
plt.show()

There is a clear inverse relationship. The customers who fly the most (Left) almost always fly alone (Low Companion Ratio). Conversely, the customers on the Right fly less often but travel in groups. Also, High-frequency flyers (Left) are not necessarily flying far; they might be taking short, frequent domestic hops. The "Long-Haul" customers are a distinct group, likely taking fewer but much more expensive intercontinental trips. The recency-months map is predominantly blue/green, indicating a generally active customer base. In the flight_trend_scope we can notice the blue patches on the Left (Frequent Flyers). This suggests some of your best customers are slowing down their activity (Negative Slope)

In [ ]:
plot_som_diagnostics(som_behave, customer[features_behave].values)

##### 7.2.6.2 SOMS with K-means

In [ ]:
labels_som_behave = run_som_kmeans(som_behave, customer[features_behave].values, n_clusters=3)

##### 7.2.6.3 SOMS with Hierarchical clustering

In [ ]:
run_som_hierarchical(som_behave, n_clusters=3)

#### Behavioral segmentation results

In [ ]:
X_behave = customer[features_behave]
kmeans_final = KMeans(n_clusters=4, init='k-means++', n_init=20, random_state=42)
labels_kmeans = kmeans_final.fit_predict(X_behave)

metrics_kmeans = get_model_metrics(
    df=X_behave,
    labels=labels_kmeans,
    model_name="K-Means (k=5)",
    perspective="Behavioral"
)
results_behave.append(metrics_kmeans)


# --- 2. Hierarchical (Agglomerative) - Ward, k=5 ---

#hc_final = AgglomerativeClustering(n_clusters=5, metric='euclidean', linkage='ward')
#labels_hc = hc_final.fit_predict(X_value)
#metrics_hc = get_model_metrics(
#    df=X_value,
#    labels=labels_hc,
#    model_name="Hierarchical (Ward, k=5)",
#    perspective="Value"
#)
#results_value.append(metrics_hc)


# --- 3. DBSCAN  --- SO TEM DOIS CLUSTERS

#metrics_dbscan = get_model_metrics(
    #df=X_behave,
    #labels=labels_behave, 
    #model_name="DBSCAN (eps=0.45)",
    #perspective="Behavioral"
#)
#results_behave.append(metrics_dbscan)


# --- 4. HDBSCAN  ---
metrics_hdb = get_model_metrics(
    df=X_behave,
    labels=labels_hdb_behave,
    model_name="HDBSCAN (min_cluster=200)",
    perspective="Behavioral"
)
results_behave.append(metrics_hdb)


# --- 5. Mean Shift ---
metrics_ms = get_model_metrics(
    df=X_behave,
    labels=df_behave_meanshift['Cluster_MS_behave'],
    model_name="Mean Shift",
    perspective="Behavioral"
)
results_behave.append(metrics_ms)


# --- 6. Gaussian Mixture ---

labels_value_gmm = gmm_model_behave.predict(X_behave)

metrics_gmm = get_model_metrics(
    df=X_behave,
    labels=labels_value_gmm,
    model_name="GMM (k=7)",
    perspective="Behavioral"
)
results_behave.append(metrics_gmm)

In [ ]:
df_compare_behave = pd.DataFrame(results_behave)
df_compare_behave = df_compare_behave.sort_values(by='Silhouette Score', ascending=False)
df_compare_behave

### 7.3 Seasonality Segmentation Perspective

In [ ]:
features_season = clustering_perspectives['Seasonality_Segmentation']

In [ ]:
results_season = []

#### 7.3.1 K-means with Hierarchical clustering

In [ ]:
df_season_kmeans_hc = customer[features_season].copy()
df_season_kmeans_hc

In [ ]:
df_season_sample = df_season_kmeans_hc.sample(n=5000, random_state=42)

In [ ]:
r2_scores_season = {}

df_season_sample = df_season_kmeans_hc.sample(n=5000, random_state=42)
r2_scores_season['kmeans'] = func.get_r2_scores(df_season_sample, df_season_kmeans_hc.columns.tolist(), kmeans)

for linkage in ['complete', 'average', 'single', 'ward']: # 4 different versions of HC
    r2_scores_season[linkage] = func.get_r2_scores(
        df_season_sample,                 # data
        df_season_kmeans_hc.columns.tolist(),   # features of perspective
        # use HClust, changing the linkage at each iteration
        hierarchical.set_params(linkage=linkage)
    )

In [ ]:
r2_scores_season = pd.DataFrame(r2_scores)
r2_scores_season

In [ ]:
r2_scores_df_season = pd.DataFrame(r2_scores)

r2_scores_df_season.plot.line(figsize=(10,7))

plt.title("Demographic Variables:\nR² plot for various clustering methods\n", fontsize=21)
plt.legend(title="Cluster methods", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13)
plt.show()

Again, when the topic of we are talking about is comparison of clustering algorithms, K-Means outperforms other methods, achieving the highest explained variance across all k values. Regarding the number of clusters, we identify a elbow at k=4 one more time. It's not a surprise the 4 cluster since we have 4 diferents seasonality.

In [ ]:
df_sample_season_viz = df_season_kmeans_hc.sample(n=3000, random_state=42)

In [ ]:
func.visualize_silhouette_graf(df_sample_season_viz, range_clusters=range(2, 11))

The silhouette score for k=3 is the highest, whats its strange, because does not confirm that 4 clusters is the optimal choice for this dataset. In the future we should try to use k = 3 and k = 4 just to see what generates better homogeneous distribution.

In [ ]:
kmeans_season = KMeans(
    n_clusters=4,
    init='k-means++',
    n_init=20,
    random_state=42
)

In [ ]:
season_labels = kmeans_season.fit_predict(df_season_kmeans_hc)
season_labels

In [ ]:
print(pd.Series(season_labels).value_counts().sort_index())

With k=3, the distribution showed a large peak in one of the clusters, which didn’t make sense. With k=4, the distribution was more balanced. Considering the graph of the number of flights per month and per year, there were two major peaks that likely correspond to the two largest clusters, while the other clusters represented smaller seasonal peaks.

#### 7.3.2 DBSCAN

In [ ]:
k_season = plot_k_distance(customer, features_season)

In [ ]:
labels_season, n_clus_season, dist_season = get_dbscan(
    customer, 
    features_season, 
    eps=0.35,
    min_samples=9
)

dbscan_results['Season'] = {'Clusters': n_clus_season, 'Distribution': dist_season}

DBSCAN shows us again that it is not a good option for this dataset, because of a large pick in one cluster, and a poor distribution of the other clusters.

#### 7.3.3 HDBSCAN

In [ ]:
labels_hdb_season, n_hdb_season, dist_hdb_season = func.get_hdbscan(
    customer, 
    features_season, 
    min_cluster_size=200, 
    min_samples=9         
)

HDBSCAN goes in the same way of the DBSCAN, shows us again that it is not a good option for this dataset, because of a large pick in one cluster, and a poor distribution of the other clusters.

#### 7.3.4 Mean-shift

In [ ]:
df_season_meanshift= customer.copy()
df_season_meanshift['Cluster_MS_season'] = get_meanshift(customer, features_season)

In [ ]:
print(df_season_meanshift.groupby('Cluster_MS_season')[features_season].mean())
print("\nCluster Sizes:")
print(df_season_meanshift.groupby('Cluster_MS_season').size())

#### 7.3.5 Gaussian Mixture Model

In [ ]:
df_season_gaussian_mix_model= customer.copy()

In [ ]:
get_n_components(df_season_gaussian_mix_model, features_season)

The **Full Covariance** structure is strictly superior to the Diagonal structure for this dataset.
* **Magnitude of Error:** The Full Covariance model achieves extremely low criterion scores, reaching approximately **-150,000**. In contrast, the Diagonal model performs significantly worse, with scores remaining positive (lowest around **12,000**).
* **Implication:** The massive difference in scores confirms that the features are intrinsically linked. The Diagonal model's assumption of independence is invalid for this data.

**Component Selection (Full Covariance)**
The Full Covariance plot exhibits a sharp **"Elbow" pattern** that supports the selection of 3 clusters:
* **Precipitous Drop ($K=1$ to $K=2$):** The AIC/BIC values plummet vertically, indicating the first split captures the massive bulk of variance.
* **The "Elbow" ($K=3$):** A significant improvement continues up to **3 components**, reaching a score near **-145,000**. While $K=4$ offers a slight improvement, the curve begins to flatten noticeably after $K=3$, indicating the primary structures are identified.
* **Diminishing Returns:** Beyond $K=3$, the rate of improvement slows down drastically compared to the initial drops. The model stabilizes, suggesting that adding more complexity yields marginal statistical gain.

**Decision:** A **Full Covariance GMM with 3 components** is selected as the optimal model.

In [ ]:
gmm_model_season, df_concat_season = fit_gmm_segmentation(df_season_gaussian_mix_model, features_season, k=3, rsq_func=get_rsq)

In [ ]:
analyze_gmm_uncertainty(gmm_model_season, df_season_gaussian_mix_model, features_season, threshold=0.99)

**Uncertainty Analysis**
* **Threshold:** A rigorous confidence threshold of **99%** was applied to evaluate the clarity of the seasonal segments.
* **Results:** The model successfully classified **14,951 customers** with clear confidence. Only **38 customers** (**0.3%**) were flagged as uncertain. This exceptionally low uncertainty indicates that the identified seasonal behaviors are highly distinct, with almost no ambiguity in customer assignment even at a very strict threshold.


#### 7.3.6 SOM

##### 7.3.6.1 Component planes and U-matrix

In [ ]:
grid_size = 12
features_to_use = features_season

# Inicializar e Treinar
som_season = MiniSom(x=grid_size, y=grid_size, input_len=len(features_to_use),
                  topology='hexagonal', sigma=1.0, learning_rate=0.5,
                  random_seed=42)

som_season.pca_weights_init(customer[features_to_use].values)
som_season.train_batch(customer[features_to_use].values, 20000, verbose=True)

weights_season = som_season.get_weights()

In [ ]:
n_plots = len(features_season)
n_cols = 3 
n_rows = math.ceil(n_plots / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 9))
axes = axes.flatten()

# 4. Loop through features and plot
for i, feature_name in enumerate(features_season):
    # Extract weights for this specific feature
    feature_weights = weights_season[:, :, i]
    
    # Plot the hexagon map
    # Using Spectral_r: Red = High Values, Blue = Low Values
    plot_hexagons(feature_weights, som_season, axes[i], label=feature_name, cmap=cm.Spectral_r)

# 5. Hide the empty subplot (since we have 5 plots for 6 spaces)
for j in range(n_plots, len(axes)):
    axes[j].axis('off')

plt.suptitle("Component Planes: Seasonality Perspective", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
plot_som_diagnostics(som_season, customer[features_season].values)

##### 7.3.6.2 SOMS with K-means

In [ ]:
labels_som_season = run_som_kmeans(som_season, customer[features_season].values, n_clusters=4)

##### 7.3.6.3 SOMS with Hierarchical clustering

In [ ]:
run_som_hierarchical(som_season, n_clusters=3)

#### Seasonality Segmentation Results

In [ ]:
X_season = customer[features_season]
kmeans_final = KMeans(n_clusters=4, init='k-means++', n_init=20, random_state=42)
labels_kmeans = kmeans_final.fit_predict(X_season)

metrics_kmeans = get_model_metrics(
    df=X_season,
    labels=labels_kmeans,
    model_name="K-Means (k=4)",
    perspective="Seasonality"
)
results_season.append(metrics_kmeans)


# --- 2. Hierarchical (Agglomerative) - Ward, k=5 ---

#hc_final = AgglomerativeClustering(n_clusters=5, metric='euclidean', linkage='ward')
#labels_hc = hc_final.fit_predict(X_value)
#metrics_hc = get_model_metrics(
#    df=X_value,
#    labels=labels_hc,
#    model_name="Hierarchical (Ward, k=5)",
#    perspective="Value"
#)
#results_value.append(metrics_hc)


# --- 3. DBSCAN  ---

metrics_dbscan = get_model_metrics(
    df=X_season,
    labels=labels_season, 
    model_name="DBSCAN (eps=0.35)",
    perspective="Seasonality"
)
results_season.append(metrics_dbscan)


# --- 4. HDBSCAN  ---
metrics_hdb = get_model_metrics(
    df=X_season,
    labels=labels_hdb_season,
    model_name="HDBSCAN (min_cluster=200)",
    perspective="Seasonality"
)
results_season.append(metrics_hdb)


# --- 5. Mean Shift ---
metrics_ms = get_model_metrics(
    df=X_season,
    labels=df_season_meanshift['Cluster_MS_season'],
    model_name="Mean Shift",
    perspective="Seasonality"
)
results_season.append(metrics_ms)


# --- 6. Gaussian Mixture ---

labels_season_gmm = gmm_model_season.predict(X_season)

metrics_gmm = get_model_metrics(
    df=X_season,
    labels=labels_season_gmm,
    model_name="GMM (k=4)",
    perspective="Seasonality"
)
results_season.append(metrics_gmm)

In [ ]:
df_compare_season = pd.DataFrame(results_season)
df_compare_season = df_compare_behave.sort_values(by='Silhouette Score', ascending=False)
df_compare_season

### 7.4 Final Clustering Solution

In [ ]:

kmeans_value = KMeans(
    n_clusters=4,
    init='k-means++',
    n_init=20,
    random_state=42
)
#value_labels = kmeans_value.fit_predict(df_value)
#value_labels

In [ ]:
#customer['value_cluster'] = value_labels

In [ ]:
kmeans_behave = KMeans(
    n_clusters=3,
    init='k-means++',
    n_init=20,
    random_state=42
)
#behave_labels = kmeans_behave.fit_predict(df_behave)
#behave_labels

In [ ]:
#customer['behavioral_cluster'] = behave_labels

In [ ]:
#print(f"Behavioral clusters: {customer['value_cluster'].nunique()}")
#print(f"Preference clusters: {customer['behavioral_cluster'].nunique()}")

In [ ]:
#crosstab = pd.crosstab(customer['value_cluster'], customer['behavioral_cluster'])
#crosstab

reavaliar os graficos seguintes apos o features selection

In [ ]:
cols_value = df_value.columns.tolist() 
temp_df = customer[cols_value].copy()
temp_df['Cluster'] = customer['value_cluster']

# Agrupar e calcular média
profile_value = temp_df.groupby('Cluster').mean().T

# Desenhar
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(profile_value, center=0.5, annot=True, cmap="RdBu_r", fmt=".2f", ax=ax)
ax.set_title("Perfilagem: Value Clusters (K=4)")
plt.show()

In [ ]:
cols_behave = df_behave.columns.tolist()
temp_df_b = customer[cols_behave].copy()
temp_df_b['Cluster'] = customer['behavioral_cluster']

# Agrupar e calcular média
profile_behave = temp_df_b.groupby('Cluster').mean().T

# Desenhar
fig, ax = plt.subplots(figsize=(8, 8)) # Mais alto porque deve ter mais variáveis
sns.heatmap(profile_behave, center=0.5, annot=True, cmap="RdBu_r", fmt=".2f", ax=ax)
ax.set_title("Perfilagem: Behavioral Clusters (K=4)")
plt.show()

### 7.5 Merge clusters using Hierarchical Clustering

In [ ]:
df_centroids = customer.groupby(['value_cluster', 'behavioral_cluster'])[fs_metric_features].mean()
df_centroids

In [ ]:
print(f"\nCentroids shape: {df_centroids.shape}")

In [ ]:
# Fit hierarchical clustering with distance tracking
hclust = AgglomerativeClustering(
    linkage='ward',
    metric='euclidean',
    distance_threshold=0,
    n_clusters=None
)
hclust_labels = hclust.fit_predict(df_centroids)

# Build linkage matrix for dendrogram
# (Adapted from scikit-learn documentation)
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

for i, merge in enumerate(hclust.children_):
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            current_count += 1 
        else:
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

linkage_matrix = np.column_stack([
    hclust.children_,
    hclust.distances_,
    counts
]).astype(float)

print("Linkage matrix ready for dendrogram!")

In [ ]:
fig = plt.figure(figsize=(12, 6))

y_threshold = 3.0 # Can adjust this!

dendrogram(
    linkage_matrix,
    labels=df_centroids.index,
    color_threshold=y_threshold,
    above_threshold_color='k'
)

plt.axhline(y=y_threshold, color='r', linestyle='--', linewidth=2, label=f'Cut ahreshot {y_threshold}')

plt.title(f'Hierarchical Clustering on 12 Centroids\n(Ward Linkage)', fontsize=16)
plt.ylabel('Euclidean Distance')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
n_final_clusters = 4

hclust_final = AgglomerativeClustering(
    linkage='ward',
    n_clusters=n_final_clusters
)
hclust_final_labels=hclust_final.fit_predict(df_centroids)
hclust_final_labels

In [ ]:
df_centroids['merged_labels'] = hclust_final_labels
df_centroids

In [ ]:
cluster_mapper = df_centroids['merged_labels'].to_dict()
cluster_mapper  

In [ ]:
customer['merged_labels'] = customer.apply(
    lambda row: cluster_mapper[(row['value_cluster'], row['behavioral_cluster'])],
    axis=1
)
customer

In [ ]:
print(customer['merged_labels'].value_counts().sort_index())

In [ ]:
# Quick distinctiveness check (R²)
r2_merged = get_rsq(customer, fs_metric_features, 'merged_labels')
print(f"\nMerged clustering R²: {r2_merged:.4f}")

## 8. Profiling

#### 8.1 Metric Features

In [ ]:
# Get centroids for ALL segments
all_segments = sorted(customer['merged_labels'].unique())
df_all_centroids = customer.groupby('merged_labels')[fs_metric_features].mean()

# Create figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# LEFT PLOT: Parallel coordinates for all clusters
plt.sca(ax1)
pd.plotting.parallel_coordinates(
    df_all_centroids.reset_index(),
    'merged_labels',
    color=sns.color_palette('tab10', n_colors=len(all_segments)),
    ax=ax1
)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5, linewidth=1.5)
ax1.set_title('Parallel Coordinates: All Segments', fontsize=14, fontweight='bold')
ax1.set_ylabel('Standardized Value', fontsize=12)
ax1.set_xlabel('Features', fontsize=12)
ax1.tick_params(axis='x', rotation=45)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
ax1.legend(title='Cluster', loc='best', title_fontsize=11)
ax1.grid(True, alpha=0.3)

# RIGHT PLOT: Cluster sizes (ordered by cluster number)
cluster_sizes = customer['merged_labels'].value_counts().sort_index()
colors = sns.color_palette('tab10', n_colors=len(all_segments))

ax2.bar(cluster_sizes.index.astype(str), cluster_sizes.values, color=colors)
ax2.set_xlabel('Number of Customers', fontsize=12)
ax2.set_ylabel('Cluster', fontsize=12)
ax2.set_title('Cluster Sizes', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
sns.set_style("whitegrid")

all_segments = sorted(customer['merged_labels'].unique())
df_all_centroids = customer.groupby('merged_labels')[fs_metric_features].mean().T

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 9))

# Heatmap
sns.heatmap(
    df_all_centroids,
    ax=ax1,
    cmap="YlGnBu",
    annot=True,
    fmt=".2f",
    cbar_kws={"shrink": 0.7}
)
ax1.set_title('Heatmap: All Segments', fontsize=16, fontweight='bold')
ax1.set_ylabel('Metric Features', fontsize=13)
ax1.set_xlabel('Clusters', fontsize=13)
ax1.tick_params(axis='x', rotation=45)

# Cluster sizes
cluster_sizes = customer['merged_labels'].value_counts().sort_index()
colors = sns.color_palette('tab10', n_colors=len(all_segments))

sns.barplot(
    x=cluster_sizes.index.astype(str),
    y=cluster_sizes.values,
    palette=colors,
    ax=ax2
)
ax2.set_title('Cluster Sizes', fontsize=16, fontweight='bold')
ax2.set_xlabel('Clusters', fontsize=13)
ax2.set_ylabel('Number of Customers', fontsize=13)

for i, val in enumerate(cluster_sizes.values):
    ax2.text(i, val + max(cluster_sizes.values)*0.01, str(val), ha='center', fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
df_seg_4 = customer[customer["merged_labels"]==1]
RFM_counts = (
    df_seg_4.groupby("RFM_score").size().reset_index(name="count").sort_values("count",ascending=False)
)
RFM_counts
#customer["RFM_score"].nunique()


##### Behavioral and Value-based perspective clusters

**Cluster 0: The "Ghosted Us" Exes**

**Analysis**
The defining characteristic of this group is the extremely high **`recency_months` (4.41)** score. This outlier indicates a significant lapse in time since their last interaction. All other engagement metrics (flights, spending) are negatively correlated.

> **Business Impact**
> These customers are effectively dormant or churned. A final, low-cost re-engagement campaign is recommended, but further investment is likely wasted as they have disengaged from the brand.

---

**Cluster 1: The "Economy Class Heroes"**

**Analysis**
This is the **largest segment (6,397 customers)**. While their **`Income`** is lower (-0.68) compared to the Elite group, their travel frequency (**`NumFlights`**) and distance metrics are positive and robust. They fly as often as the VIPs but likely on a budget.

> **Business Impact**
> They provide the essential operational volume for the airline. They are loyal but price-sensitive. The focus should be on volume-based rewards and reliability rather than luxury services.

---

**Cluster 2: The "Stay-at-Home" Club**

**Analysis**
This group scores negatively across almost all engagement metrics, particularly **`NumFlights` (-1.28)** and **`DistanceKM` (-1.29)**. Despite having average income levels, they simply do not travel with your airline.

> **Business Impact**
> These are low-yield customers. They do not justify significant marketing spend. They may be seasonal travelers or users who use the program for non-flight partnerships.

---

**Cluster 3: The "Free Lunch" Hunters**

**Analysis**
A statistical anomaly (only 36 customers), but behaviorally distinct. They show the highest **`PointsUtilizationRate` (1.21)** combined with the lowest **`Tenure` (-1.02)**.

> **Business Impact**
> These appear to be new users who joined specifically to redeem rewards or exploit a sign-up bonus ("churning"). They are cashing out value without contributing to revenue. This segment should be monitored for fraud or policy loopholes.

---

**Cluster 4: The "Wolf of Wall Street" VIPs**

**Analysis**
This segment represents your high-value customers. They exhibit the highest **`Income` (0.95)** and **`Customer Lifetime Value` (1.06)** scores. Furthermore, they are highly active travelers (high `NumFlights` and `DistanceKM`) with solid loyalty (`Tenure`).

> **Business Impact**
> These are the primary revenue drivers. The strategy should focus on premium retention, exclusive perks, and high-touch service to prevent churn to competitors.

***

#### 8.2 Non Metric Features

Here in profile we can see the distribution of the non metric features by cluster, but we can conclude that just education and marital status are relevant to the segments.

In [ ]:
for i in Demographic_Segmentation:
    pd.crosstab(customer['merged_labels'], customer[i], normalize='index').plot.bar(
    stacked=True,
    figsize=(10, 6)
    )
    plt.title(f"{i} Distribution by Merged Segment", fontsize=14)
    plt.xlabel("Merged Segment")
    plt.ylabel("Proportion")
    plt.legend(title=i, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

##### Demographic Deep-Dive: Connecting Behavior to Identity
**1. Education is the Strongest Predictor of Value**

The distinction between VIPs and the mass market is not just about how much they fly, but their professional background.

The Data: The high-value Cluster 4 is almost exclusively composed of holders of a Bachelor's degree. In contrast, the mass-market Cluster 1 has a significant portion of users with "College" education.

Conclusion: Our most profitable customers are white-collar professionals. This validates the "Wolf of Wall Street" persona, these are business travelers whose flights are likely paid for by their companies or supported by higher professional salaries.

**2. The "Cluster 3" Anomaly: The Montreal Glitch**

We previously identified Cluster 3 as the "Free Lunch Hunters" (high points use, low loyalty). The demographic data reveals exactly where this happened.

The Data: While other clusters are distributed across the country (Toronto, Vancouver, etc.), Cluster 3 is overwhelmingly concentrated in Quebec and specifically Montreal.

Conclusion: This suggests the high rate of points churning wasn't a general problem, but likely the result of a specific local campaign, partnership, or loophole in the Quebec market. It’s a regional issue, not a national one.

**3. Stability Drives Loyalty**

There is a clear correlation between life stability and airline loyalty.

The Data: Cluster 4 (VIPs) has the highest proportion of Married customers. Meanwhile, Cluster 3 (The Hunters) has the largest percentage of Single users.

Conclusion: Our VIPs are established individuals with families, valuing consistency and comfort. The "Hunters" are likely younger, single urbanites (specifically in Montreal) who have the flexibility to chase deals and switch brands aggressively.

## 9. Merged Clusters Visualisation

In [ ]:
customer['merged_labels'].value_counts().sort_index()

In [ ]:
len(fs_metric_features)

### 9.1. PCA

In [ ]:
pca = PCA(n_components=2, random_state=42)
pca_result = pca.fit_transform(customer[fs_metric_features])

pca_result.shape

In [ ]:
customer['pca_1'] = pca_result[:, 0]
customer['pca_2'] = pca_result[:, 1]

# Variance explained
var_explained = pca.explained_variance_ratio_
print(f"PCA Variance Explained:")
print(f"  PC1:      {var_explained[0]:.1%}")
print(f"  PC2:      {var_explained[1]:.1%}")
print("="*20)
print(f"  Total:    {var_explained.sum():.1%}")

Here we are going to see the lossin 2D PCA projection

In [ ]:
plot_pca_variance(data=customer,features=fs_metric_features)

In [ ]:
plot_pca_clusters(df=customer,label_col='merged_labels',pc1_col='pca_1',pc2_col='pca_2',var_explained=var_explained,title='2D PCA Projection of 7 Customer Segments')

In [ ]:
pc_corr = customer[fs_metric_features+['pca_1', 'pca_2']].corr().round(3)
pc_corr = pc_corr.loc[fs_metric_features, ['pca_1', 'pca_2']]
pc_corr

In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=metric_features
)

print("PCA Component Loadings (Feature Contributions):\n")
print(loadings.round(3))

### 9.2. t-SNE

In [ ]:
two_dim = TSNE(n_components=2, random_state=42, perplexity=20, n_iter_without_progress=500).fit_transform(customer[fs_metric_features])

In [ ]:
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=customer['merged_labels'], colormap='Set2', figsize=(15,10))
plt.show()

t-SNE with pca

In [ ]:
two_dim = TSNE(random_state=42, perplexity=30).fit_transform(pca_result)

pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=customer['merged_labels'], colormap='Set2', figsize=(15,10))
plt.show()

### 9.3. UMAP

In [ ]:
##todo